In [1]:
import pandas as pd

In [2]:
def fix_smiles(smi_in):
    return smi_in.strip('[]').strip("'")

In [3]:
df_list = []
for split in ["random","scaffold"]:
    for task in ["st","mt"]:
        for i in range(0,10):
            dirname = f"data/Sol{i:03d}"
            if task == "st":
                df_test = pd.read_csv(f"{dirname}/{split}_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_ST/test_preds.csv")
            else:
                df_test = pd.read_csv(f"{dirname}/{split}_mt_test_{i:03d}.csv")
                df_pred = pd.read_csv(f"{dirname}/{split}_result_MT/test_preds.csv")
            df_pred['SMILES'] = df_pred.smiles.apply(fix_smiles)
            df_pred.drop('smiles',axis=1,inplace=True)
            df_pred.rename(columns={"Sol": "Sol_prob"},inplace=True)
            df_test.Sol = df_test.Sol.astype(int)
            df_combo = pd.concat([df_test[["SMILES","Name","Sol"]],df_pred[["Sol_prob"]]],axis=1)
            df_combo['Sol_pred'] = [1 if p > 0.5 else 0 for p in df_combo.Sol_prob]
            df_combo['cv_cycle'] = i
            df_combo['split'] = split
            df_combo['method'] = f"chemprop_{task}"
            df_combo = df_combo[["cv_cycle","split","method","SMILES","Name","Sol","Sol_prob","Sol_pred"]]
            df_list.append(df_combo)

In [4]:
df_out = pd.concat(df_list)
df_out.to_csv("Sol_classification_preds.csv",index=False)

In [5]:
df_out

,cv_cycle,split,method,SMILES,Name,Sol,Sol_prob,Sol_pred
0,0,random,chemprop_st,CCc1cc(NCc2cccc(OCCOC)c2)ncn1,Mol465,1,0.733507,1
1,0,random,chemprop_st,CC(C)c1cnc(COc2ccc(Br)nc2)o1,Mol1224,1,0.750664,1
2,0,random,chemprop_st,Cc1ccc(C)c(C(=O)Nc2cnc3c(c2)c(C)nn3C)c1,Mol673,0,0.446773,0
3,0,random,chemprop_st,Cc1nn(-c2ccc(C(=O)O)cc2C#Cc2ccccc2)c(C)c1C,Mol207,1,0.571375,1
4,0,random,chemprop_st,C=CCn1cnc2c(Cl)cc(-c3ccc(C(=O)NC4CC4)o3)cc21,Mol1078,0,0.253248,0
...,...,...,...,...,...,...,...,...
213,9,scaffold,chemprop_mt,COc1cc(Nc2ncc3c(n2)-c2ccc(Cl)cc2C(c2c(F)cccc2O...,Mol1155,1,0.225647,0
214,9,scaffold,chemprop_mt,CCCCC1=NC2(CCCC2)C(=O)N1Cc1ccc(-c2ccccc2-c2nn[...,Mol340,0,0.102379,0
215,9,scaffold,chemprop_mt,N#Cc1ccc2nc([C@H]3CCN(C4CCC4)C3)[nH]c2c1,Mol1451,1,0.529411,1
216,9,scaffold,chemprop_mt,N#CC1(NC(=O)CN2CCOCC2)CCCCCC1,Mol1211,1,0.977073,1


### Concatenate the results

In [ ]:
df_chemprop = pd.read_csv("Sol_classification_preds.csv")
df_lightgbm = pd.read_csv("lightgbm_classifciation_results.csv")
pd.concat([df_chemprop, df_lightgbm]).to_csv("Sol_classification_ChemProp_LightGBM.csv",index=False)